In [36]:
import numpy as np
import bilby 
import pycbc 
import sys
import matplotlib.pyplot as plt

import zuko
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn


In [2]:
import river.data
from river.data.datagenerator import DataGeneratorBilbyFD
from river.data.dataset import DatasetStrainFD
#import river.data.utils as datautils
from river.data.utils import *

from river.models import embedding
from river.models.utils import *
from river.models.embedding.pca import project_strain_data_FDAPhi
from river.models.embedding.conv import EmbeddingConv1D, EmbeddingConv2D

In [3]:
import pickle

In [4]:
from sklearn.decomposition import IncrementalPCA

In [5]:
import sklearn.decomposition 

In [6]:
Nsample = 3
injection_parameters_all = generate_BNS_injection_parameters(
        Nsample,
        a_max=0.1,
        d_min=10,
        d_max=100,
        d_power=2)


Nsample_template = 10000
injection_parameters_template = generate_BNS_injection_parameters(
        Nsample_template,
        a_max=0.1,
        d_min=10,
        d_max=100,
        d_power=2)

In [7]:
source_type = 'BNS'
detector_names = ['H1', 'L1', 'V1'] 
duration = 32
f_low = 20
f_ref = 20
sampling_frequency = 2048
waveform_approximant = 'IMRPhenomPv2_NRTidal'
parameter_names = PARAMETER_NAMES_PRECESSINGBNS_BILBY
PSD_type = 'bilby_default' #'zero_noise'
use_sealgw_detector = True

In [8]:
data_generator = DataGeneratorBilbyFD(source_type,
            detector_names, 
            duration, 
            f_low, 
            f_ref, 
            sampling_frequency, 
            waveform_approximant, 
            parameter_names,
            PSD_type=PSD_type,
            use_sealgw_detector=use_sealgw_detector)
'''
data_template_generator = DataGeneratorBilbyFD(source_type,
            detector_names, 
            duration, 
            f_low, 
            f_ref, 
            sampling_frequency, 
            waveform_approximant, 
            parameter_names,
            PSD_type='zero_noise',
            use_sealgw_detector=use_sealgw_detector)
'''

15:47 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using bilby_default PSDs to generate data.


"\ndata_template_generator = DataGeneratorBilbyFD(source_type,\n            detector_names, \n            duration, \n            f_low, \n            f_ref, \n            sampling_frequency, \n            waveform_approximant, \n            parameter_names,\n            PSD_type='zero_noise',\n            use_sealgw_detector=use_sealgw_detector)\n"

In [9]:
data_generator.inject_signals(injection_parameters_all, Nsample)
#data_template_generator.inject_signals(injection_parameters_template, Nsample_template)

15:47 bilby WARNING : The injected signal has a duration in-band of 94.9s, but the data for detector H1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 94.9s, but the data for detector L1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 94.9s, but the data for detector V1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 120.7s, but the data for detector H1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 120.7s, but the data for detector L1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 120.7s, but the data for detector V1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 90.8s, but the data for detector H1 has a duration of 32s
15:47 bilby WARNING : The injected signal has a duration in-band of 90.8s, but the data for detector L1 has a durat

Injecting 0-th signal, 0.0% done
Injecting 1-th signal, 33.33% done
Injecting 2-th signal, 66.67% done


In [10]:
data_generator.numpy_starins()
#data_template_generator.numpy_starins()

#data_generator.whiten_strains()
#data_template_generator.scale_strains()

In [11]:
#n_components=10
#ipca_gen = embedding.pca.IPCAGenerator(data_template_generator.data['strains'], n_components, detector_names, decomposition='exp_unwrap')

In [12]:
#modelutils.save_model('IPCAgenerator.pickle', ipca_gen)

In [13]:
ipca_gen = load_model('../scripts/ipca_models/IPCA_BNSFD_10000to500_ExpUnwrap_fixtc.pickle')



Model loaded from ../scripts/ipca_models/IPCA_BNSFD_10000to500_ExpUnwrap_fixtc.pickle


In [14]:
#ipca_gen.pca_dict['H1']['phase'].singular_values_

In [15]:
ds = DatasetStrainFD(data_dict=data_generator.data, parameter_names=PARAMETER_NAMES_PRECESSINGBNS_BILBY)

In [30]:
def get_condition_2proj(embedding_proj, embedding_noproj, theta, strain, psd, detector_names, ipca_gen, device, downsample_rate):
    inputs_proj = project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen).to(device)
    inputs_noproj = project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen, project=False, dim=2, downsample_rate=downsample_rate).to(device)
    theta = theta.to(device)

    embedding_out_proj = embedding_proj(inputs_proj)
    embedding_out_noproj = embedding_noproj(inputs_noproj)
    condition = torch.cat((embedding_out_proj, embedding_out_noproj), -1)

    return condition

In [40]:
type(embedding_proj) in [EmbeddingConv1D]

True

In [31]:
embedding_proj = EmbeddingConv1D(ndet=3, ncomp=1, nout=128, use_psd = True, middle_channel = 2,
                                 kernel_size=1, stride=1, padding=0, dilation=1)

In [32]:
embedding_noproj = EmbeddingConv2D(ndet=3, ncomp=1, nout=128, use_psd = True, middle_channel = 20,
                                   kernel_size=1, stride=1, padding=0, dilation=1)

In [33]:
device = 'cpu'
theta, strain, psd = ds[0:2]
downsample_rate= 4

In [35]:
get_condition_2proj(embedding_proj, embedding_noproj, theta, strain, psd, detector_names, ipca_gen, device, downsample_rate).shape

torch.Size([2, 256])

In [40]:
device = 'cpu'
downsample_rate = 4
inputs_proj = project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen)#.to(device)
inputs_noproj = project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen, project=False,downsample_rate=4, dim=1)#.to(device)
theta = theta#.to(device)

In [36]:
inputs_proj.shape

torch.Size([2, 9, 500])

In [37]:
inputs_noproj.shape

torch.Size([2, 1, 9, 8033])

In [38]:
embedding_proj(inputs_proj).shape

torch.Size([2, 128])

In [41]:
embedding_noproj(inputs_noproj.unsqueeze(-2)).shape

torch.Size([2, 128])

In [ ]:
theta, strain, psd = ds[0:2]

In [ ]:
get_condition_2proj(embedding_proj, embedding_noproj, theta, strain, psd, detector_names, ipca_gen, device='cpu', downsample_rate=4)

In [ ]:
proj_angle = np.unwrap(np.angle(strain), axis=-1)


In [ ]:
proj_angle.shape

In [ ]:
proj_angle = np.unwrap(np.angle(strain), axis=-1)
plt.plot(proj_angle[0][0])
plt.plot(proj_angle[0][1])
plt.plot(proj_angle[0][2])

In [ ]:
np.array(list(data_dict['strains'].values()))

In [ ]:
def project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen):
    strain_amp = np.abs(strain)
    strain_phi = np.unwrap(np.angle(strain) , axis=-1)
    n_components = ipca_gen.n_components

    output_amp = []
    output_phi = []
    output_psd = []
    for i,detname in enumerate(detector_names):
        output_amp.append(ipca_gen.project(strain_amp[:,i,:], detname, 'amplitude'))
        output_phi.append(ipca_gen.project(strain_phi[:,i,:], detname, 'phase'))
        output_psd.append(ipca_gen.project(psd[:,i,:], detname, 'amplitude'))
    output_amp = torch.from_numpy(np.array(output_amp))
    output_phi = torch.from_numpy(np.array(output_phi))
    output_psd = torch.from_numpy(np.array(output_psd))

    return torch.cat((output_amp, output_phi, output_psd)).movedim(0,1).float()


In [ ]:
theta, strain, psd = ds[0:2]
psd.shape

In [ ]:
theta, strain, psd = ds[0:2]

detector_names = data_generator.detector_names
aa = project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen)

In [ ]:
aa.shape

In [ ]:
channel = 'strains'
detname = 'H1'
test_strain = np.abs(data_generator.data[channel][detname])


In [ ]:
test_strain[0]

In [ ]:
test_strain.shape

In [ ]:
ipca_gen.project(test_strain[0:3], detname, 'amplitude')

In [ ]:
bank_H1 = data_template_generator.data['strains']['H1'].copy()

In [ ]:
u,s,vt = np.linalg.svd(bank_H1.T, full_matrices=False)

In [ ]:
padding = 0
dialation = 1
kernel_size = 1
stride = 1


In [ ]:
int((256 + 2*padding - dialation*(kernel_size-1) - 1)/stride) + 1

In [ ]:
class EmbeddingConv(nn.Module):
    def __init__(self, ndet, ncomp, nout, use_psd = True):
        super().__init__()
        self.ncomp = ncomp
        self.nout = nout
        if use_psd:
            self.nchannel = 3*ndet # strains(2) + PSD (1)
        else:
            self.nchannel = 2*ndet

        self.middle_channel = 512
        self.cnn1 = nn.Sequential(
            nn.Conv1d(in_channels=self.nchannel, out_channels=self.middle_channel, kernel_size=1, stride=1),
            nn.BatchNorm1d(self.middle_channel),
            #nn.MaxPool1d(kernel_size=2)
        )
        
        self.cnn2 = nn.Sequential(
            nn.Conv1d(in_channels=self.middle_channel, out_channels=self.middle_channel, kernel_size=1, stride=1),
            nn.BatchNorm1d(self.middle_channel),
            #nn.MaxPool1d(kernel_size=2)
        )

        self.cnn3 = nn.Sequential(
            nn.Conv1d(in_channels=self.middle_channel, out_channels=self.middle_channel, kernel_size=1, stride=1),
            nn.BatchNorm1d(self.middle_channel),
            #nn.MaxPool1d(kernel_size=2)
        )
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(self.middle_channel*self.ncomp, self.nout)


    def forward(self, x):
        # x : [batch_size, channel (det_123, amp/phase) = 2*ndet, length (number of samples)]
        bs,_,_  = x.shape 
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.dropout(x).reshape((bs,-1))
        output = self.linear(x)

        return output

In [ ]:
Net_emb = EmbeddingConv(ndet=3, ncomp=n_components, nout=128)

In [ ]:
theta, strain, psd = ds[0:2]
test_input = project_strain_data_FDAPhi(strain, psd, detector_names, ipca_gen)

In [ ]:
embedding_out = Net_emb(test_input)

In [ ]:
test_input.shape

In [ ]:
theta.shape

In [ ]:
flow = zuko.flows.NSF(features=17, context=128, transforms=3, hidden_features=(64, 64))


In [ ]:
loss = -flow(embedding_out).log_prob(theta).mean()

In [ ]:
loss